# Information Retrival Project



# 1- Positional Index Implementation


1.1 Load Documents

In [54]:
import json
import hazm
import math

with open('IR_data_news_12k.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

content_dataset, url_dataset, title_dataset = [], [], []
for _, data in json_data.items():
    content_dataset.append(data["content"])
    url_dataset.append(data["url"])
    title_dataset.append(data["title"])

normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
positional_index = {}
tokens = []

1.2 Preprocess Documents

In [55]:
def preprocess_documents():
    stop_words = hazm.stopwords_list()
    punctuations = [')', '(', '>', '<', "؛", "،", '{', '}', "؟", ':', "–", '»', '"', '«', '[', ']', '"', '+', '=', '?',
                    '/',
                    '//', '\\', '|', '!', '%', '&', '*', '$', '#', '؟', '*', '.', '_', '']
    for i in range(len(content_dataset)):
        content_dataset[i] = normalizer.normalize(content_dataset[i])
        first_word_tokens_list = hazm.word_tokenize(content_dataset[i])
        second_word_tokens_list = []
        final_word_tokens_list = []
        for word_token in first_word_tokens_list:
            if word_token not in stop_words and word_token not in punctuations:
                second_word_tokens_list.append(word_token)
        for j in range(len(second_word_tokens_list)):
            stem_lammatize = lemmatizer.lemmatize(second_word_tokens_list[j])
            if lemmatizer not in final_word_tokens_list:
                if stem_lammatize != '':
                    final_word_tokens_list.append(stem_lammatize)
        for word_token in final_word_tokens_list:
            tokens.append((word_token, i))
    # Sort Tokens
    tokens.sort(key=lambda a: a[0])


preprocess_documents()

1.3 Sort Tokens and Create Positional Index

In [56]:
def create_positional_index():
    # create postings list
    last_token = None
    last_doc_id = None
    for i in range(len(tokens)):
        positional_postings = {}
        positions = []
        token_i, doc_id_i = tokens[i]
        if token_i != last_token:
            positional_postings[doc_id_i] = (positions, 0, None)
            positional_index[token_i] = (1, positional_postings, None)
        else:
            if doc_id_i != last_doc_id:
                positional_postings = positional_index[token_i][1]
                positional_postings[doc_id_i] = (positions, 0)
                positional_index[token_i] = (positional_index[token_i][0] + 1, positional_postings)
        last_token = token_i
        last_doc_id = doc_id_i
    # create positional index
    for i in range(len(content_dataset)):
        word_tokens_list = hazm.word_tokenize(content_dataset[i])
        for position, word_token in enumerate(word_tokens_list):
            stem_lammatize = lemmatizer.lemmatize(word_token)
            if stem_lammatize in positional_index.keys():
                positional_postings = positional_index[stem_lammatize][1]
                if i in positional_postings.keys():
                    positions = positional_postings[i][0]
                    positions.append(position)
                    positional_postings[i] = (positions, positional_postings[i][1] + 1)
                    positional_index[stem_lammatize] = (positional_index[stem_lammatize][0], positional_postings)


create_positional_index()

Output:

In [58]:
term = 'سجاد'
print("Number of frequencies in all documents:" + str(positional_index[term][0]))
for index, (docID, positional_postings) in enumerate(positional_index[term][1].items()):
    if index > 20:
        break
    positions = positional_postings[0]
    print("docId=" + str(docID) + " positions=" + str(positions))

ranked_result_docs = {}

Number of frequencies in all documents:301
docId=1 positions=[7]
docId=3 positions=[215]
docId=16 positions=[189]
docId=52 positions=[224]
docId=62 positions=[437]
docId=65 positions=[184]
docId=77 positions=[51]
docId=116 positions=[357]
docId=128 positions=[260]
docId=140 positions=[654, 1351]
docId=232 positions=[126]
docId=274 positions=[139]
docId=277 positions=[537, 621]
docId=278 positions=[205]
docId=291 positions=[84]
docId=315 positions=[6]
docId=350 positions=[65, 70]
docId=376 positions=[376]
docId=378 positions=[6, 196, 396]
docId=435 positions=[624]
docId=449 positions=[589, 656]


1.4 Preprocess Query and Retrieving Documents

In [59]:
def retrieve_documents(query):
    doc_id_with_last_phrase_position = {}

    # Preprocess Query
    split_query = query.split(" ")
    see_str = False
    phrase_str = ''
    phrase_list = []
    not_list = []
    and_list = []
    for split_word in split_query:
        if '"' in split_word:
            if not see_str:
                phrase_str += split_word.split('"')[1]
                see_str = True
            else:
                phrase_str += " " + split_word.split('"')[0]

                see_str = False
                phrase_list.append(phrase_str)
                phrase_str = ''
        elif see_str:
            phrase_str += " " + split_word

        elif '!' in split_word:
            not_list.append(split_word.split('!')[1])

        else:
            and_list.append(split_word)
    and_list_tokens = []
    for i in range(len(and_list)):
        word = normalizer.normalize(and_list[i])
        word = lemmatizer.lemmatize(word)
        and_list_tokens.append(word)
    not_list_tokens = []
    for i in range(len(not_list)):
        word = normalizer.normalize(not_list[i])
        word = lemmatizer.lemmatize(word)
        not_list_tokens.append(word)
    phrase_list_tokens = []
    for i in range(len(phrase_list)):
        phrase = normalizer.normalize(phrase_list[i])
        phrase_list_tokens.append(phrase)

    # intersect
    for i in range(len(and_list_tokens)):
        if and_list_tokens[i] in positional_index.keys():
            result_docs = positional_index[and_list_tokens[i]][1].keys()
            for doc in result_docs:
                if doc not in ranked_result_docs.keys():
                    ranked_result_docs[doc] = 1
                else:
                    ranked_result_docs[doc] = ranked_result_docs[doc] + 1
    # not
    for i in range(len(not_list_tokens)):
        if not_list_tokens[i] in positional_index.keys():
            result_docs = positional_index[not_list_tokens[i]][1].keys()
            for doc in result_docs:
                if doc in ranked_result_docs.keys():
                    ranked_result_docs.pop(doc)

    # phrase
    for i in range(len(phrase_list_tokens)):
        result_docs = []
        phrase_words = phrase_list_tokens[i].split(" ")
        for word in phrase_words:
            word_lemmatize = lemmatizer.lemmatize(word)
            if word_lemmatize in positional_index.keys():
                result_docs.append(positional_index[word_lemmatize][1])

        for first_doc_id in result_docs[0].keys():
            for second_doc_id in result_docs[1].keys():
                if first_doc_id == second_doc_id:
                    positions_list = []
                    first_positions_list = result_docs[0][first_doc_id][0]
                    second_positions_list = result_docs[1][second_doc_id][0]
                    for position1 in first_positions_list:
                        for position2 in second_positions_list:
                            if position1 + 1 == position2:
                                positions_list.append(position2)
                    if len(positions_list) != 0:
                        doc_id_with_last_phrase_position[first_doc_id] = positions_list
        if len(doc_id_with_last_phrase_position) != 0:
            for j in range(2, len(result_docs)):
                new_doc_id_with_last_position = {}
                for doc_id in result_docs[j].keys():
                    new_positions_list = []
                    if doc_id in doc_id_with_last_phrase_position.keys():
                        positions_list = result_docs[j][doc_id][0]
                        for position1 in doc_id_with_last_phrase_position[doc_id]:
                            for position2 in positions_list:

                                if position1 + 1 == position2:
                                    new_positions_list.append(position2)
                    if len(new_positions_list) != 0:
                        new_doc_id_with_last_position[doc_id] = new_positions_list
                doc_id_with_last_phrase_position = new_doc_id_with_last_position

        for doc_id in doc_id_with_last_phrase_position.keys():
            if doc_id not in ranked_result_docs.keys():
                ranked_result_docs[doc_id] = 1
            else:
                ranked_result_docs[doc_id] = ranked_result_docs[doc_id] + 1
    return doc_id_with_last_phrase_position, and_list_tokens


ranked_result_docs = dict(sorted(ranked_result_docs.items(), key=lambda x: x[1], reverse=True))



1.5 Show Result with Title and Sentences

In [60]:
def show_result(doc_id_with_last_phrase_position, and_list_tokens):
    for i, (doc_id, number) in enumerate(ranked_result_docs.items()):
        positions = []
        sentences = []
        for word in and_list_tokens:
            if word in positional_index.keys():
                if doc_id in positional_index[word][1].keys():
                    for position in positional_index[word][1][doc_id][0]:
                        positions.append(position)
        if doc_id in doc_id_with_last_phrase_position.keys():
            for position in doc_id_with_last_phrase_position[doc_id]:
                positions.append(position)

        if i < 5:
            print("Document " + str(doc_id) + ":")

            print(title_dataset[doc_id])

            doc_words = hazm.word_tokenize(content_dataset[doc_id])
            for count, word in enumerate(doc_words):
                if count in positions:
                    sentence = ''
                    for k in range(count, -1, -1):
                        if doc_words[k] != '.':
                            if k in positions:
                                positions.remove(k)
                            sentence = doc_words[k] + ' ' + sentence
                        else:
                            break
                    for j in range(count + 1, len(doc_words)):
                        if doc_words[j] != '.':
                            if j in positions:
                                positions.remove(j)
                            sentence = sentence + ' ' + doc_words[j]
                        else:
                            sentence += '.'
                            break

                    sentences.append(sentence)
            for num, sentence in enumerate(sentences):
                print("\n        Sentence " + str(num + 1) + ":")
                print(sentence + '\n')

        else:
            break

Output:

In [61]:
# query = 'باشگاه‌های فوتسال آسیا'
# query = 'باشگاه‌های فوتسال !آسیا'
query = '"سهمیه المپیک"'
# query = '"لیگ برتر" !والیبال'
# query = 'مایکل !جردن'
documents_with_last_phrase_position, tokens_list = retrieve_documents(query)
show_result(documents_with_last_phrase_position, tokens_list)

Document 86:
نشست خبری جشنواره یکصد برای انتخاب برترین‌های قرن ورزش ایران برگزار شد

        Sentence 1:
رشته‌هایی که در آسیا مدال گرفتند یا سهمیه المپیک  گرفتند سه نامزد معرفی می‌کنند.

Document 164:
رستمیان: رنکینگ ما در کسب سهمیه المپیک مهم است/حضور در جام جهانی قطعی نیست

        Sentence 1:
وی گفت : امتیاز این مسابقات خیلی مهم است چون از روی رنکینگ سهمیه المپیک  توزیع می‌شود ، باید بتوانیم جایگاه خود را در رنکینگ حفظ کنیم.


        Sentence 2:
از همین حالا امتیاز مسابقات و رنکینگ ما در توزیع سهمیه المپیک  پاریس تاثیرگذار است.

Document 228:
گلخندان: مدال فروغی در المپیک مشوق خوبی برای همه بود/ نمی‌توان برای بازی‌های آسیایی قول داد

        Sentence 1:
وی در خصوص کسب سهمیه المپیک  از طریق رنکینگ جهانی ، گفت : از سال گذشته فدراسیون جهانی برای سهمیه المپیک رنکینگ را مدنظر قرار داد.


        Sentence 2:
کسانی که رنکینگ جهانی داشته_باشند با قوانین خاصی می‌توانند سهمیه المپیک  بگیرند اما همیشه شرکت در هر مسابقه جهانی در رنکینگ تاثیر داشته_است.

Document 458:
میراسماعیلی: دلسوزی دبیر ن